In [142]:
!pip install Lifetimes

In [143]:
# Importing necessary libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

In [144]:
df = pd.read_csv('/content/DFORE115CustDate.zip', compression='zip', low_memory=False)

In [145]:
df['txnMonth']= pd.to_datetime(df['txnMonth'], format='%Y%m')
df.vendAmt= df.vendAmt/100.00

In [146]:
df.isna().sum()

vendor                   0
branch_id            11318
client_account_id        0
payment_type             0
namedUser                0
txnMonth                 0
vends                    0
vendAmt                  0
dtype: int64

In [147]:
df.head()

,vendor,branch_id,client_account_id,payment_type,namedUser,txnMonth,vends,vendAmt
0,DI,002,200380,LaundryCard,30300001000000294201,2021-06-01,11,21.85
1,DI,074,7407672,App,286343,2020-07-01,23,27.00
2,DI,021,7318306,LaundryCard,30300001000000248729,2021-06-01,6,4.50
3,DI,051,5108741,App,413240,2020-05-01,4,8.00
4,DI,072,7230203,LaundryCard,30300001000000265604,2021-06-01,14,22.35


In [148]:
df.vends.describe(percentiles=[.95])

count    5.763556e+06
mean     1.196267e+01
std      1.542621e+01
min      1.000000e+00
50%      8.000000e+00
95%      3.600000e+01
max      4.598000e+03
Name: vends, dtype: float64

In [149]:
for v in ['vends', 'vendAmt']:
  p95= df[v].describe(percentiles=[.05,.25,.50,.75,.95]).loc['95%']
  df[v] = np.where(df[v] >p95, p95,df[v])


In [150]:
df.columns

Index(['vendor', 'branch_id', 'client_account_id', 'payment_type', 'namedUser',
       'txnMonth', 'vends', 'vendAmt'],
      dtype='object')

In [151]:
# Printing the details of the dataset
maxdate = df['txnMonth'].dt.date.max()
mindate = df['txnMonth'].dt.date.min()
unique_cust = df['namedUser'].nunique()
tot_quantity = df['vends'].sum()
tot_sales = df['vendAmt'].sum()

print(f"The Time range of transactions is: {mindate} to {maxdate}")
print(f"Total number of unique customers: {unique_cust}")
print(f"Total Vends: {tot_quantity}")
print(f"Total Vend Amt $: {tot_sales}")

The Time range of transactions is: 2019-07-01 to 2021-06-01
Total number of unique customers: 1350470
Total Vends: 63026023.0
Total Vend Amt $: 91331592.98


# Aggregate Model

The most simplest and the oldest method of computing CLV is this Aggregate/Average method. This assumes a constant average spend and churn rate for all the customers.

This method does not differentiate between customers and produces a single value for CLV at an overall Level. This leads to unrealistic estimates if some of the customers transacted in high value and high volume, which ultimately skews the average CLV value.

The General Formula for calculating CLV is:

CLV = ((Average Sales X Purchase Frequency) / Churn)x Profit Margin
Where,
Average Sales = TotalSales/Total no. of orders

Purchase Frequency = Total no. of orders/Total unique customers

Retention rate = Total no. of orders greater than 1/ Total unique customers

Churn = 1 - Retention rate

Profit Margin = Based on business context [Leave it out for now]

## 2 years- App

In [152]:
df1= df.loc[df.payment_type=='App']

In [153]:
# Transforming the data to customer level for the analysis
c1 = df1.groupby('namedUser').agg({'txnMonth':lambda x: (x.max() - x.min()).days, 
                                                   'vends': lambda x: len(x),
                                                  'vendAmt': lambda x: sum(x)})

c1.columns = ['Age', 'Frequency', 'TotalSales']
c1.head()

,Age,Frequency,TotalSales
namedUser,,,
1,0,1.0,0.50
100,701,37.0,235.98
1000,701,24.0,746.00
100000,182,7.0,28.50
1000000,0,1.0,4.00


In [157]:
c1.shape

(1126311, 3)

In [158]:

# Calculating the necessary variables for CLV calculation
Average_sales = round(np.mean(c1['TotalSales']),2)
print(f"Average Vend Amt: ${Average_sales}")

Purchase_freq = round(np.mean(c1['Frequency']), 2)
print(f"Vend Frequency: {Purchase_freq}")

Retention_rate = c1[c1['Frequency']>1].shape[0]/c1.shape[0]
churn = round(1 - Retention_rate, 2)
print(f"Retention: {Retention_rate}")
print(f"Churn: {churn}")

Average Vend Amt: $62.07
Vend Frequency: 4.0
Retention: 0.5955140276531082
Churn: 0.4


In [159]:
# Calculating the CLV
Profit_margin = 1 

CLV = round(((Average_sales * Purchase_freq/churn)) * Profit_margin, 2)
print(f"The Customer Lifetime Value (CLV) for each customer is: ${CLV}")

The Customer Lifetime Value (CLV) for each customer is: $620.7


## 2 years- Laundry Card

In [160]:
df1= df.loc[df.payment_type=='LaundryCard']
# Transforming the data to customer level for the analysis
c1 = df1.groupby('namedUser').agg({'txnMonth':lambda x: (x.max() - x.min()).days, 
                                                   'vends': lambda x: len(x),
                                                  'vendAmt': lambda x: sum(x)})

c1.columns = ['Age', 'Frequency', 'TotalSales']

# Calculating the necessary variables for CLV calculation
Average_sales = round(np.mean(c1['TotalSales']),2)
print(f"Average Vend Amt: ${Average_sales}")

Purchase_freq = round(np.mean(c1['Frequency']), 2)
print(f"Vend Frequency: {Purchase_freq}")

Retention_rate = c1[c1['Frequency']>1].shape[0]/c1.shape[0]
churn = round(1 - Retention_rate, 2)
print(f"Retention: {Retention_rate}")
print(f"Churn: {churn}")
# Calculating the CLV
Profit_margin = 1 

CLV = round(((Average_sales * Purchase_freq/churn)) * Profit_margin, 2)
print(f"The Customer Lifetime Value (CLV) for each customer is: ${CLV}")

Average Vend Amt: $95.56
Vend Frequency: 5.61
Retention: 0.7389085425969959
Churn: 0.26
The Customer Lifetime Value (CLV) for each customer is: $2061.89


## 1 Year- App

In [161]:
df1= df.loc[df.payment_type=='App']
df1= df1.loc[df1.txnMonth>= '2020-07-01']
# Transforming the data to customer level for the analysis
c1 = df1.groupby('namedUser').agg({'txnMonth':lambda x: (x.max() - x.min()).days, 
                                                   'vends': lambda x: len(x),
                                                  'vendAmt': lambda x: sum(x)})

c1.columns = ['Age', 'Frequency', 'TotalSales']
# Calculating the necessary variables for CLV calculation
Average_sales = round(np.mean(c1['TotalSales']),2)
print(f"Average Vend Amt: ${Average_sales}")

Purchase_freq = round(np.mean(c1['Frequency']), 2)
print(f"Vend Frequency: {Purchase_freq}")

Retention_rate = c1[c1['Frequency']>1].shape[0]/c1.shape[0]
churn = round(1 - Retention_rate, 2)
print(f"Retention: {Retention_rate}")
print(f"Churn: {churn}")
# Calculating the CLV
Profit_margin = 1 

CLV = round(((Average_sales * Purchase_freq/churn)) * Profit_margin, 2)
print(f"The Customer Lifetime Value (CLV) for each customer is: ${CLV}")

Average Vend Amt: $60.72
Vend Frequency: 3.79
Retention: 0.6330305876749662
Churn: 0.37
The Customer Lifetime Value (CLV) for each customer is: $621.97


## 1 Year- LaundryCard

In [162]:
df1= df.loc[df.payment_type=='LaundryCard']
df1= df1.loc[df1.txnMonth>= '2020-07-01']
# Transforming the data to customer level for the analysis
c1 = df1.groupby('namedUser').agg({'txnMonth':lambda x: (x.max() - x.min()).days, 
                                                   'vends': lambda x: len(x),
                                                  'vendAmt': lambda x: sum(x)})

c1.columns = ['Age', 'Frequency', 'TotalSales']
# Calculating the necessary variables for CLV calculation
Average_sales = round(np.mean(c1['TotalSales']),2)
print(f"Average Vend Amt: ${Average_sales}")

Purchase_freq = round(np.mean(c1['Frequency']), 2)
print(f"Vend Frequency: {Purchase_freq}")

Retention_rate = c1[c1['Frequency']>1].shape[0]/c1.shape[0]
churn = round(1 - Retention_rate, 2)
print(f"Retention: {Retention_rate}")
print(f"Churn: {churn}")
# Calculating the CLV
Profit_margin = 1 

CLV = round(((Average_sales * Purchase_freq/churn)) * Profit_margin, 2)
print(f"The Customer Lifetime Value (CLV) for each customer is: ${CLV}")

Average Vend Amt: $82.0
Vend Frequency: 4.7
Retention: 0.740121683197639
Churn: 0.26
The Customer Lifetime Value (CLV) for each customer is: $1482.31


# Cohort Model
Instead of simply assuming all the customers to be one group, we can try to split them into multiple groups based and calculate the CLV for each group. This model overcomes the major drawback of the simple Aggregate model which assumes the entire customers as a single group. This is called Cohort model.

The main assumption of this model is that, customers within a cohort spend similarly.

The most common way to group customers into cohorts is by the start date of a customer, typically by month. The best choice will depend on the customer acquisition rate, seasonality of business, and whether additional customer information can be used.

In this case, I am grouping them into different cohorts by their start month. So, I will get 12 cohorts of customers (Jan-Dec).

## Cohort Model- App

In [163]:
data_clv= df.loc[df.payment_type=='App']
data_clv= data_clv.loc[data_clv.txnMonth>= '2020-07-01']

In [164]:
# Transforming the data to customer level for the analysis
customer = data_clv.groupby('namedUser').agg({'txnMonth':lambda x: x.min().month,  
                                                   'vends': lambda x: len(x),
                                                  'vendAmt': lambda x: np.sum(x)})

customer.columns = ['Start_Month', 'Frequency', 'TotalSales']

In [165]:
months= [7,8,9,10,11,12,1,2,3,4,5,6]
Monthly_CLV = []

for m in months:
  customer_m = customer[customer['Start_Month']==m]
  Average_sales = round(np.mean(customer_m['TotalSales']),2)
  Purchase_freq = round(np.mean(customer_m['Frequency']), 2)
  Retention_rate = customer_m[customer_m['Frequency']>1].shape[0]/customer_m.shape[0]
  churn = round(1 - Retention_rate, 2)
  CLV = round(((Average_sales * Purchase_freq/churn)) * Profit_margin, 2)
  Monthly_CLV.append(CLV)

In [166]:
monthly_clv = pd.DataFrame(zip(months, Monthly_CLV), columns=['Months', 'CLV'])
display(monthly_clv.style.background_gradient())

,Months,CLV
0,7,5004.500000
1,8,1534.700000
2,9,1261.670000
3,10,957.130000
4,11,687.110000
5,12,492.080000
6,1,382.410000
7,2,295.030000
8,3,192.140000
9,4,125.480000


In [167]:
monthly_clv.mean()

Months      6.500000
CLV       916.941667
dtype: float64

## Cohort Model- LaundryCard

In [168]:
data_clv= df.loc[df.payment_type=='LaundryCard']
data_clv= data_clv.loc[data_clv.txnMonth>= '2020-07-01']
# Transforming the data to customer level for the analysis
customer = data_clv.groupby('# Creating the summary data using summary_data_from_transaction_data function
summary = lifetimes.utils.summary_data_from_transaction_data(data_clv, 'CustomerID', 'InvoiceDate', 'TotalSales' )
summary = summary.reset_index()
summary.head()').agg({'txnMonth':lambda x: x.min().month,  
                                                   'vends': lambda x: len(x),
                                                  'vendAmt': lambda x: np.sum(x)})

customer.columns = ['Start_Month', 'Frequency', 'TotalSales']
months= [7,8,9,10,11,12,1,2,3,4,5,6]
Monthly_CLV = []

for m in months:
  customer_m = customer[customer['Start_Month']==m]
  Average_sales = round(np.mean(customer_m['TotalSales']),2)
  Purchase_freq = round(np.mean(customer_m['Frequency']), 2)
  Retention_rate = customer_m[customer_m['Frequency']>1].shape[0]/customer_m.shape[0]
  churn = round(1 - Retention_rate, 2)
  CLV = round(((Average_sales * Purchase_freq/churn)) * Profit_margin, 2)
  Monthly_CLV.append(CLV)
  monthly_clv = pd.DataFrame(zip(months, Monthly_CLV), columns=['Months', 'CLV'])

display(monthly_clv.style.background_gradient())

SyntaxError: ignored

In [ ]:
monthly_clv.mean()

# BG/NBD Model (with Gamma-Gamma extension)
BG/NBD stands for Beta Geometric/Negative Binomial Distribution.

This is one of the most commonly used probabilistic model for predicting the CLV. This is an alternative to the Pareto/NBD model, which is also one of the most used method in CLV calculations. For the sake of this case, we are going to focus only on BG/NBD model, but the steps are similar if you want to try it for Pareto/NBD.

To be precise, both the BG/NBD and Pareto/NBD model actually tries to predict the future transactions of each customer. It is then combined with Gamma-Gamma model, which then adds the monetary aspect of the customer transaction and we finally get the customer lifetime value (CLV).

The BG/NBD model has few assumptions:

When a user is active, number of transactions in a time t is described by Poisson distribution with rate lambda.

Heterogeneity in transaction across users (difference in purchasing behavior across users) has Gamma distribution with shape parameter r and scale parameter a.

Users may become inactive after any transaction with probability p and their dropout point is distributed between purchases with Geometric distribution.

Heterogeneity in dropout probability has Beta distribution with the two shape parameters alpha and beta.

Transaction rate and dropout probability vary independently across users.

These are some of the assumptions this model considers for predicting the future transactions of a customer.

The model technically fits the distribution to the historic data and learn the distribution parameter and then use them to predict future transactions of a customer.

We don't need to worry about carrying out this complex probabilistic model by ourself. There is a Python package called Lifetimes which makes our life much simpler. This package is primarily built to aid customer lifetime value calculations, predicting customer churn, etc. It has all the major models and utility functions which are needed for CLV calculations.

In this case, we are going to use just that. Let's jump into the coding.

In [ ]:
# Importing the lifetimes package
import lifetimes

First we need to create a summary table from the transactions data. The summary table is nothing but RFM table. (RFM - Recency, Frequency and Monetary value)

For this we can use summary_data_from_transactions_data function in lifetimes package. What it does is, it aggregates the transaction level data into customer level and calculates the frequency, recency, T and monetary_value for each customer.

frequency - the number of repeat purchases (more than 1 purchases)
recency - the time between the first and the last transaction
T - the time between the first purchase and the end of the transaction period
monetary_value - it is the mean of a given customers sales value
NOTE: If you closely look at the definition of recency and T, you can find that, the actual value of recency should be (T - recency), because the definition of recency is how recent a customer made a transaction with the business.

In [ ]:
df.columns

In [ ]:
df1= df.loc[df.payment_type=='App']

In [ ]:
# Creating the summary data using summary_data_from_transaction_data function
summary = lifetimes.utils.summary_data_from_transaction_data(df1, 'namedUser', 'txnMonth', 'vendAmt' )
summary = summary.reset_index()
summary.head()

In [ ]:
summary['frequency'].plot(kind='hist', bins=50)
print(summary['frequency'].describe())
print("---------------------------------------")
one_time_buyers = round(sum(summary['frequency'] == 0)/float(len(summary))*(100),2)
print("Percentage of customers purchase the item only once:", one_time_buyers ,"%")

In [ ]:

# Fitting the BG/NBD model
bgf = lifetimes.BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(summary['frequency'], summary['recency'], summary['T'])

In [ ]:
bgf.summary

The above table shows the estimated distribution parameter values from the historical data. The model now uses this to predict the future transactions and the customer churn rate.

So, let's say you want to know whether a customer is now alive or not (or predict customer churn) based on the historical data. The lifetimes package provides a way to accomplish that task. You can use:

1. model.conditional_probability_alive(): This method computes the probability that a customer with history (frequency, recency, T) is currently alive.

2. plot_probabilty_alive_matrix(model): This function from lifetimes.plotting will help to visually analyze the relationship between recency & frequency and the customer being alive.

In [ ]:
# Compute the customer alive probability
summary['probability_alive'] = bgf.conditional_probability_alive(summary['frequency'], summary['recency'], summary['T'])
summary.head(10)

In [ ]:
# Visual representation of relationship between recency and frequency
from lifetimes.plotting import plot_probability_alive_matrix

fig = plt.figure(figsize=(15,10))
plot_probability_alive_matrix(bgf)

The probabilty of being alive is calculated based on the recency and frequency of a customer. So,

If a customer has bought multiple times (frequency) and the time between first & last transaction is high (recency), then his/her probability being alive is high.
Similarly, if a customer has less frequency (bought once or twice) and the time between first & last transaction is low (recency), then his/her probability being alive is high.
Next thing we can do with this trained model is to predict the likely future transactions for each customer. You can use:

1. model.conditional_expected_number_of_purchases_up_to_time(): Calculate the expected number of repeat purchases up to time t for a randomly chosen individual from the population (or the whole population), given they have purchase history (frequency, recency, T).

In [ ]:
#Predict future transaction for the next 30 days based on historical dataa
t = 30
summary['pred_num_txn'] = round(bgf.conditional_expected_number_of_purchases_up_to_time(t, summary['frequency'], summary['recency'], summary['T']),2)
summary.sort_values(by='pred_num_txn', ascending=False).head(10).reset_index()

In [ ]:
# Checking the relationship between frequency and monetary_value
return_customers_summary = summary[summary['frequency']>0]
print(return_customers_summary.shape)
return_customers_summary.head()

In [ ]:
# Checking the relationship between frequency and monetary_value
return_customers_summary[['frequency', 'monetary_value']].corr()

In [ ]:
# Modeling the monetary value using Gamma-Gamma Model
ggf = lifetimes.GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(return_customers_summary['frequency'],
       return_customers_summary['monetary_value'])

In [ ]:
# Summary of the fitted parameters
ggf.summary

In [ ]:
# Calculating the conditional expected average profit for each customer per transaction
summary = summary[summary['monetary_value'] >0]
summary['exp_avg_sales'] = ggf.conditional_expected_average_profit(summary['frequency'],
                                       summary['monetary_value'])
summary.head()

In [ ]:
print(f"Expected Average Sales: {summary['exp_avg_sales'].mean()}")
print(f"Actual Average Sales: {summary['monetary_value'].mean()}")

In [ ]:
# Predicting Customer Lifetime Value for the next 30 days
summary['predicted_clv'] =      ggf.customer_lifetime_value(bgf,
                                                               summary['frequency'],
                                                               summary['recency'],
                                                               summary['T'],
                                                               summary['monetary_value'],
                                                               time=1,     # lifetime in months
                                                               freq='D',   # frequency in which the data is present(T)      
                                                               discount_rate=0.01) # discount rate
summary.head()

In [ ]:
summary['manual_predicted_clv'] = summary['pred_num_txn'] * summary['exp_avg_sales']
summary.head()

In [ ]:
# CLV in terms of profit (profit margin is 5%)
profit_margin = 1
summary['CLV'] = summary['predicted_clv'] * profit_margin
summary.head()

In [ ]:
summary['CLV'].describe()